In [1]:
%matplotlib inline
import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')


AntsPy version = 0.5.4
SimpleITK version = 2.5.0


In [2]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\CHEEZYJEEZY\Desktop\MRIdata_preprocessing


In [3]:
test_examples = [
    'BRAINIX_NIFTI_FLAIR.nii.gz',
    'BRAINIX_NIFTI_Output_3D_File.nii.gz',
    'BRAINIX_NIFTI_ROI.nii.gz',
    'BRAINIX_NIFTI_T1.nii.gz',
    'BRAINIX_NIFTI_T2.nii.gz'
]

test_example = test_examples[0]

In [4]:
raw_img_path = os.path.join(BASE_DIR, 'NifT', test_example)
raw_img_ants = ants.image_read(raw_img_path, reorient= 'IAL')

explore_3D_array(arr = raw_img_ants.numpy(), cmap = 'nipy_spectral')

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

In [5]:
transformed = ants.denoise_image(raw_img_ants, shrink_factor= 5)

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(),
    arr_after= transformed.numpy()
)

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

In [6]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk, 'RPS')
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)
print(raw_img_sitk_arr.shape)
explore_3D_array(arr = raw_img_sitk_arr,
                 cmap = 'nipy_spectral')

(22, 288, 288)


interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

Denoising using SITK

In [7]:
transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)

transformed = sitk.LiThreshold(transformed, 0,1)

head_mask = transformed

explore_3D_array_comparison(
    arr_before = sitk.GetArrayFromImage(raw_img_sitk),
    arr_after = sitk.GetArrayFromImage(head_mask),
)

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

Bias Correction(Intensity Normalization using SITK)

In [8]:
shrinkFactor = 4
inputImage = raw_img_sitk

inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension())
maskImage = sitk.Shrink( head_mask, [shrinkFactor]* inputImage.GetDimension())

bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

corrected = bias_corrector.Execute(inputImage, maskImage)

In [ ]:
log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
corrected_image_full_resolution = raw_img_sitk / sitk.Exp(log_bias_field)

explore_3D_array_comparison(
    sitk.GetArrayFromImage(raw_img_sitk),
    sitk.GetArrayFromImage(corrected_image_full_resolution),
    cmap = 'viridis',
)

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

Image Registration using SITK

In [ ]:
template_img_path = os.path.join(BASE_DIR, 'Template', 'TEMPLATE_MNI152_T1_1mm_brain.nii.gz' )
template_img_ants = ants.image_read(template_img_path, reorient = 'IAL')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=90, description='SLICE', max=181), Output()), _dom_classes=('widget-inte…

In [13]:
from antspynet.utilities import brain_extraction
prob_brain_mask = brain_extraction(raw_img_ants,modality='t1', verbose = True)

Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Brain extraction:  renormalize probability mask to native space.


In [14]:
print(prob_brain_mask)

explore_3D_array(prob_brain_mask.numpy())

ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (22, 288, 288)
	 Spacing    : (6.0, 0.7986, 0.7986)
	 Origin     : (113.655, -109.7964, -36.4214)
	 Direction  : [-0.0239 -0.0017 -0.9997 -0.072   0.9974  0.      0.9971  0.072  -0.024 ]



interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

Generate final mask

In [15]:
brain_mask = ants.get_mask(prob_brain_mask, low_thresh = 0.5)

In [ ]:
explore_3D_array_with_mask_contour(raw_img_ants.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…

In [17]:
out_folder = os.path.join(BASE_DIR, 'assets', 'testoutput')
out_folder = os.path.join(out_folder, test_example.split('.')[0])
os.makedirs(out_folder, exist_ok = True)

out_filename = add_suffix_to_filename(test_example, suffix = 'brainmaskbyDL')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR) :])
print(out_path[len(BASE_DIR) :])

\NifT\BRAINIX_NIFTI_FLAIR.nii.gz
\assets\testoutput\BRAINIX_NIFTI_FLAIR\BRAINIX_NIFTI_FLAIR_brainmaskbyDL.nii.gz


In [18]:
brain_mask.to_filename(out_path)

In [ ]:
transformed = ants.mask_image(raw_img_ants, brain_mask)
explore_3D_array(  arr = transformed.numpy())

interactive(children=(IntSlider(value=10, description='SLICE', max=21), Output()), _dom_classes=('widget-inter…